In [1]:
import os
os.chdir('/mnt/mint_files/TCC/main_code/')

In [2]:
import numpy as np
from tensorflow import keras
from pyrsgis import raster
from pyrsgis.convert import changeDimension
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from glob import glob
import rasterio as rio
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

2023-05-27 02:43:29.033572: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-27 02:43:29.033596: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [16]:
# Lê a imagem de treino e o seu gabarito

# Assign file names
mxBangalore = 'images/bangalore_data/l5_Bangalore2011_raw.tif'
builtupBangalore = 'images/bangalore_data/l5_Bangalore2011_builtup.tif'
mxHyderabad = 'images/bangalore_data/l5_Hyderabad2011_raw.tif'

# Read the rasters as array
ds1, featuresBangalore = raster.read(mxBangalore, bands='all')
ds2, labelBangalore = raster.read(builtupBangalore, bands=1)
ds3, featuresHyderabad = raster.read(mxHyderabad, bands='all')

# Print the size of the arrays
print("Bangalore Multispectral image shape: ", featuresBangalore.shape)
print("Bangalore Binary built-up image shape: ", labelBangalore.shape)
print("Hyderabad Multispectral image shape: ", featuresHyderabad.shape)

Bangalore Multispectral image shape:  (6, 2054, 2044)
Bangalore Binary built-up image shape:  (2054, 2044)
Hyderabad Multispectral image shape:  (6, 1318, 1056)


In [17]:
# Acerta os shape das imagens de treino e de gabarito

# Clean the labelled data to replace NoData values by zero
labelBangalore = (labelBangalore == 1).astype(int)

# Reshape the array to single dimensional array
featuresBangalore = changeDimension(featuresBangalore)
labelBangalore = changeDimension (labelBangalore)
featuresHyderabad = changeDimension(featuresHyderabad)
nBands = featuresBangalore.shape[1]

print("Bangalore Multispectral image shape: ", featuresBangalore.shape)
print("Bangalore Binary built-up image shape: ", labelBangalore.shape)
print("Hyderabad Multispectral image shape: ", featuresHyderabad.shape)

The "changeDimension()" function has moved to "array_to_table()" and will be deprecated in future versions. Please check the pyrsgis documentation at https://pyrsgis.readthedocs.io/en/master/ for more details.
The "changeDimension()" function has moved to "array_to_table()" and will be deprecated in future versions. Please check the pyrsgis documentation at https://pyrsgis.readthedocs.io/en/master/ for more details.
The "changeDimension()" function has moved to "array_to_table()" and will be deprecated in future versions. Please check the pyrsgis documentation at https://pyrsgis.readthedocs.io/en/master/ for more details.
Bangalore Multispectral image shape:  (4198376, 6)
Bangalore Binary built-up image shape:  (4198376,)
Hyderabad Multispectral image shape:  (1391808, 6)


In [18]:
# Split testing and training datasets
xTrain, xTest, yTrain, yTest = train_test_split(featuresBangalore, labelBangalore, test_size=0.4, random_state=0)

print(xTrain.shape)
print(yTrain.shape)

print(xTest.shape)
print(yTest.shape)

(2519025, 6)
(2519025,)
(1679351, 6)
(1679351,)


In [19]:
# Normalise the data
xTrain = xTrain / 255.0 # bits
xTest = xTest / 255.0
featuresHyderabad = featuresHyderabad / 255.0

# Reshape the data
xTrain = xTrain.reshape((xTrain.shape[0], 1, xTrain.shape[1]))
xTest = xTest.reshape((xTest.shape[0], 1, xTest.shape[1]))
featuresHyderabad = featuresHyderabad.reshape((featuresHyderabad.shape[0], 1, featuresHyderabad.shape[1]))

# Print the shape of reshaped data
print(xTrain.shape, xTest.shape, featuresHyderabad.shape)

(2519025, 1, 6) (1679351, 1, 6) (1391808, 1, 6)


In [20]:
# Escolhe as bandas para serem utilizadas. entre 0 e 5. alterar a quantidade na primeira camada da RN
# [1,2,3] -> bands 2,3,4 - RGB
# [2,3,4] -> bands 3,4,5 - Infra vermelho
# [1,4,5] -> bands 2,5,6 - Agricultura
# [0,2,3] -> bands 1,3,4 - Bathymetric
#xTrain_b = xTrain[:,:,[1,2,3]]
#xTest_b = xTest #xTest[:,:,[1,2,3]]
#featuresRecreio_b = featuresRecreio[:,:,[0,2,3]]
bandas = [
   # ["345", np.copy(xTrain[:,:,[2,3,4]]), np.copy(xTest[:,:,[2,3,4]])],
   # ["256", np.copy(xTrain[:,:,[1,4,5]]), np.copy(xTest[:,:,[1,4,5]])],
   # ["234", np.copy(xTrain[:,:,[1,2,3]]), np.copy(xTest[:,:,[1,2,3]])],
   # ["134", np.copy(xTrain[:,:,[0,2,3]]), np.copy(xTest[:,:,[0,2,3]])]
   ["allb", np.copy(xTrain), np.copy(xTest)]
]
bandas[0] # 0-> nome, 1-> treino, 2-> teste

['ball',
 array([[[0.15294118, 0.2       , 0.22745098, 0.43529412, 0.58431373,
          0.22352941]],
 
        [[0.16078431, 0.19215686, 0.2       , 0.28627451, 0.57254902,
          0.18431373]],
 
        [[0.13333333, 0.15294118, 0.24313725, 0.41176471, 0.58039216,
          0.18431373]],
 
        ...,
 
        [[0.18039216, 0.19607843, 0.17254902, 0.26666667, 0.56078431,
          0.16078431]],
 
        [[0.11764706, 0.12941176, 0.2       , 0.27058824, 0.56078431,
          0.12156863]],
 
        [[0.11764706, 0.13333333, 0.20784314, 0.30196078, 0.57647059,
          0.13333333]]]),
 array([[[0.16470588, 0.18823529, 0.2       , 0.30196078, 0.56470588,
          0.18039216]],
 
        [[0.13333333, 0.18039216, 0.2       , 0.4627451 , 0.59215686,
          0.22745098]],
 
        [[0.1254902 , 0.14509804, 0.25882353, 0.37254902, 0.57647059,
          0.16862745]],
 
        ...,
 
        [[0.14901961, 0.19215686, 0.2745098 , 0.53333333, 0.60392157,
          0.25882353]],
 
 

In [21]:
epocas = [2] #[5,10,20,25]

for ep in epocas:
    for banda in bandas:
        print(f'inicio epoca {ep}, bandas: {banda[0]}')
        modelEpoch = keras.Sequential([
            keras.layers.Flatten(input_shape=(1, 6)), #input_shape=(1, nBands)
            keras.layers.Dense(14, activation='relu'),
            keras.layers.Dense(2, activation='softmax')]) #softmax

        # Define the accuracy metrics and parameters
        modelEpoch.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

        # Run the model
        modelEpoch.fit(banda[1], yTrain, epochs=ep) # testar auamentando o número de epochs
        modelEpoch.save(f'modelos/my_model_{banda[0]}_{ep}.h5')

        # Predict for test data 
        yTestPredicted = modelEpoch.predict(banda[2])
        yTestPredicted = yTestPredicted[:,1]

        # Calculate and display the error metrics
        yTestPredicted = (yTestPredicted>0.5).astype(int)
        cMatrix = confusion_matrix(yTest, yTestPredicted, normalize='true')
        pScore = precision_score(yTest, yTestPredicted)
        rScore = recall_score(yTest, yTestPredicted)

        with open(f'metrics/model_{banda[0]}_metric_{ep}_norm.txt', 'w') as fout:
        # fout.write(u'#'+'\t'.join(str(e) for e in cMatrix.shape)+'\n')
        # cMatrix.tofile(fout)
            for line in cMatrix:
                for val in line:
                    fout.write('%.3f\t' % val)
                fout.write('\n')
            fout.write('P-Score: %.3f, R-Score: %.3f"' % (pScore, rScore))

inicio epoca 2, bandas: ball
Epoch 1/2
78720/78720 [==============================] - 53s 668us/step - loss: 0.1276 - accuracy: 0.9518
Epoch 2/2
78720/78720 [==============================] - 53s 668us/step - loss: 0.1093 - accuracy: 0.9589
